# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**: Sarah Cooke. Laura Akey, Fabien DeSilva

**ID**: sec275, lka27, fmd48

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/hw05`
   Installed OffsetArrays ───────── v1.14.1
   Installed LERC_jll ───────────── v4.0.0+0
   Installed JpegTurbo_jll ──────── v3.0.4+0
   Installed PlotUtils ──────────── v1.4.3
   Installed NetworkLayout ──────── v0.4.7
   Installed StaticArrays ───────── v1.9.8
   Installed GR_jll ─────────────── v0.73.8+0
   Installed LoggingExtras ──────── v1.1.0
   Installed MutableArithmetics ─── v1.5.2
   Installed DataFrames ─────────── v1.7.0
   Installed Libgpg_error_jll ───── v1.50.0+0
   Installed Cairo_jll ──────────── v1.18.2+1
   Installed HiGHS_jll ──────────── v1.8.1+0
   Installed GR ─────────────────── v0.73.8
   Installed Bzip2_jll ──────────── v1.0.8+2
   Installed HTTP ───────────────── v1.10.10
   Installed CodecBzip2 ─────────── v0.8.4
   Installed FFMPEG ─────────────── v0.4.2
   Installed SentinelArrays ─────── v1.4.7
   Installed MathOptInterface ───── v1.34.0
   Installed ColorSchemes ───────── v3.27.1
   Installed LZO_jll ────────────── v2.10

In [2]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

$Problem$ $Solution$

**Optimization Equation:**\
minimize (transportation costs + disposal costs)

transportation costs = cost of transporting waste from source to disposal x distance from source to disposal x amount of waste transported

disposal costs = fixed costs of operations + variable cost of disposing waste

**Variable Description:**\
w_1_lf - waste from city 1 to landfill (LF)\
w_2_lf - waste from city 2 to landfill (LF)\
w_3_lf - waste from city 3 to landfill (LF)\
w_1_mrf - waste from city 1 materials recycling facility (MRF)\
w_2_mrf - waste from city 2 materials recycling facility (MRF)\
w_3_mrf - waste from city 3 materials recycling facility (MRF)\
w_1_wte - waste from city 1 to waste to energy facility (WTE)\
w_2_wte - waste from city 2 to waste to energy facility (WTE)\
w_3_wte - waste from city 3 to waste to energy facility (WTE)\
r_mrf_lf - residuals waste from mrf to landfill\
r_mrf_wte - residuals waste from mrf to wte\
r_wte_lf - residuals waste from wte to landfill\
y_lf - operating status of the landfill\
y_mrf - operating status of the mrf\
y_wte - operating status of the wte


**Constraint Equations:**\
The residual waste from the mrf is 0.6 times the total waste from the cities that the mrf recieves, since there is a 40% recycle rate. This waste goes to both the landfill and the waste to energy plant.\
r_mrf_lf + r_mrf_wte = 0.6(w_1_mrf + w_2_mrf + w_3_mrf)

The residual waste from the wte is equal to 14% of the waste it recieves from the recylcing plant plus 16% of the waste it recieves from the cities. This waste goes to the landfill.\
r_wte_lf = 0.14(r_mrf_wte) + 0.16(w_1_wte + w_2_wte + w_3_wte)

The landfill is either turned on or off depending if it is used or not. If anything is sent there then it is on and costs money to operate. This variable will always be on as long as any waste from any city is produced.\
y_lf = {1}

The mrf is either turned on or off depending if it is used or not. If anything is sent there then it is on and costs money to operate.\
y_mrf = {0, 1}

The wte is either turned on or off depending if it is used or not. If anything is sent there then it is on and costs money to operate.\
y_wte = {0, 1}

City 1 produces 100 Mg/day of waste.\
w_1_lf + w_1_mrf + w_1_wte = 100

City 2 produces 90 Mg/day of waste.\
w_2_lf + w_2_mrf + w_2_wte = 90

City 3 produces 120 Mg/day of waste.\
w_1_lf + w_1_mrf + w_1_wte = 120


In [14]:
# Define the model
model = Model(HiGHS.Optimizer)

# Parameters where the order is [LF,MRF,WTE]
city_waste = [100, 90, 120] # Mg/day
facility_capacities = [200, 350, 210] # Mg
fixed_costs = [2000, 1500, 2500] #$/Day
tipping_costs = [50, 7, 60] # $/Mg
recycling_cost = 40 # $/Mg
transport_cost = 1.5 # $/Mg-km
distances = [
    [5, 30, 15], # city 1
    [15, 25, 10], # city 2
    [13, 45, 20] # city 3
] # City to facilities

facility_to_facility_distances = [
    [0, 32, 18], # LF to other facilities
    [32, 0, 15], # MRF to other facilities
    [18, 15, 0]  # WTE to other facilities
]

recycling_rate = 0.4
ash_fractions = [0.16, 0.14] # non-recycled, recycled

# Decision variables
@variable(model, x[1:3, 1:3] >= 0) # Waste sent from city i to facility j
@variable(model, r[1:3, 1:3] >= 0) # Residual waste from facility j to k
@variable(model, use_facility[1:3], Bin) # Whether a facility is used
print(distances)
# Objective: Minimize total cost
@objective(
    model, Min,fixed_costs[1]*use_facility[1] + tipping_costs[1]*(x[1,1]+x[2,1]+x[3,1]+r[2,1]r[3,1]) 
    + fixed_costs[2]*use_facility[2] + tipping_costs[2]*(x[1,2]+x[2,2]+x[3,2]) + recycling_rate * recycling_cost * (x[1,2]+x[2,2]+x[3,2])
    + fixed_costs[3]*use_facility[3] + tipping_costs[3]*(x[1,3]+x[2,3]+x[3,3]+r[2,3])
    + transport_cost * (distances[1,1]*x[1,1]+distances[1,2]*x[1,2]+distances[1,3]*x[1,3]
    + distances[2,1]*x[2,1]+distances[2,2]*x[2,2]+distances[2,3]*x[2,3]
    + distances[3,1]*x[3,1]+distances[3,2]*x[3,2]+distances[3,3]*x[3,3]
    + facility_to_facility_distances[3,1]*r[3,1]+facility_to_facility_distances[2,3]*r[2,3]+facility_to_facility_distances[2,1]*r[2,1])
)
print("working objective")
# Constraints
# Balance waste generated and disposed
for i in 1:3
    @constraint(model, sum(x[i, j] for j in 1:3) == city_waste[i])
end

# Capacity constraints for each facility
for j in 1:3
    @constraint(
        model,
        sum(x[i, j] for i in 1:3) + sum(r[k, j] for k in 1:3) <= facility_capacities[j] * use_facility[j]
    )
end
# Residual ash constraint for MRF 
@constraint(
    model,
    sum(r[2, 1] for k in 1:3) + sum(r[2, 3] for k in 1:3) == 0.6 * (sum(x[i, 2] for i in 1:3) + sum(x[i, 3] for i in 1:3))
)
# Commitment constraints for three variables (facilities)
@constraint(waste, commit1, !Y[1] => {W[1,1] + W[2,1] + W[3,1] == 0})
@constraint(waste, commit2, !Y[2] => {W[1,2] + W[2,2] + W[3,2] == 0})
@constraint(waste, commit3, !Y[3] => {W[1,3] + W[2,3] + W[3,3] == 0})

# Solve the model
optimize!(model)
println(model)

# Output results
println("Optimal Cost: ", objective_value(model))
for j in 1:3
    println("Facility $j Used: ", value(use_facility[j]))
end
for i in 1:3, j in 1:3
    println("Waste from City $i to Facility $j: ", value(x[i, j]))
end
for j in 1:3, k in 1:3
    println("Residual Waste from Facility $j to Facility $k: ", value(r[j, k]))
end

[[5, 30, 15], [15, 25, 10], [13, 45, 20]]

BoundsError: BoundsError: attempt to access 3-element Vector{Vector{Int64}} at index [1, 2]

## References

List any external references consulted, including classmates.